In [1]:
#import warnings
#warnings.filterwarnings('ignore') # make the notebook nicer

In [2]:
#conda install -c ambermd pytraj
#conda install -c bioconda nglview
#jupyter-nbextension enable nglview --py --sys-prefix
#%matplotlib widget

In [1]:
#interactive 
import pytraj as pt
import nglview as nv
from subprocess import call
with open("polyA.pdb", "w") as f:
    call(["./fd_helix", "abdna","aaaaaaaaaa"], stdout=f)
traj = pt.load('polyA.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

NGLWidget()

In [4]:
#conda install -c omnia openmm
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

pdb = PDBFile('polyA.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [5]:
pdb.getTopology()

<Topology; 2 chains, 20 residues, 404 atoms, 452 bonds>

In [6]:
modeller = Modeller (pdb.topology, pdb.positions)
modeller.addHydrogens (forcefield)
modeller.getTopology()

<Topology; 2 chains, 20 residues, 638 atoms, 686 bonds>

In [7]:
pdbfile.PDBFile.writeModel(modeller.topology,modeller.positions,open('polyA_with_H.pdb','w'))
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

NGLWidget()

In [8]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)

In [9]:
system = forcefield.createSystem(modeller.topology)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
#simulation.minimizeEnergy()

In [10]:
simulation.reporters.append(PDBReporter('polyA_with_H.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True,potentialEnergy=True, temperature=True))
simulation.step(3000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,6922.2353411801105,251.19802681416382
1000,6755.057710382678,297.0859240013492
1500,6432.9124269706135,313.4890700880711
2000,6241.179360203956,318.4486435460084
2500,5910.44631935772,318.989769595388
3000,5631.4193086892865,338.41419266036803


In [11]:
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

NGLWidget(count=30)

In [16]:
from time import sleep
for frame in range(0, 30):
    view.frame = frame
    # make sure to let NGL spending enough time to update coordinates
    sleep(0.5)
    view.download_image(filename='0image{}.png'.format(frame))
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(2.0)
#move images to renders/DNA_denature
import moviepy.editor as mpy
template = 'renders/DNA_denature/0image{}.png'
imagefiles = [template.format(str(i)) for i in range(0, 30)]
frame_per_second = 8
im = mpy.ImageSequenceClip(imagefiles, fps=frame_per_second)
im.write_gif('renders/my_movie.gif', fps=frame_per_second)

In [19]:
from IPython import display
display.HTML("<img src='renders/my_movie.gif'></img>")